<a href="https://colab.research.google.com/github/GiliardGodoi/tj-datasets/blob/april-24/notebooks/02_Exemplo_Pre_Processamento_Limpo_e_Normalizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "tjdatasets @ git+https://github.com/GiliardGodoi/tj-datasets@april-24"

  Cloning https://github.com/GiliardGodoi/tj-datasets (to revision april-24) to /tmp/pip-install-yacj2joj/tjdatasets_8bab761969ba46849ef2830b25dfd623
  Running command git clone --filter=blob:none --quiet https://github.com/GiliardGodoi/tj-datasets /tmp/pip-install-yacj2joj/tjdatasets_8bab761969ba46849ef2830b25dfd623
  Running command git checkout -b april-24 --track origin/april-24
  Switched to a new branch 'april-24'
  Branch 'april-24' set up to track remote branch 'april-24' from 'origin'.
  Resolved https://github.com/GiliardGodoi/tj-datasets to commit 89ccc8705835520e76eb3c5efe9d84bfbf1028d0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tjdatasets: filename=tjdatasets-0.0.3-py3-none-any.whl size=18288 sha256=7b26d528e25146a96bea2da58c7f4d123392930bb1806d38df5317b755589e21
  Stored in directory: /tmp/pip-ephem-wheel-cache-0uzj56u5

In [3]:
import pandas as pd
import re

from pathlib import Path

from tjdatasets.pipe import (
    PreProcessamentoLimpo,
    PreProcessamentoNormalizado
)
from tjdatasets.utils import save_as_parquet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
from ipywidgets import Box, Layout, Textarea, IntText, BoundedIntText

def display(df, index, left_column='conteudo', right_column='formatado'):
    # index = 9578 # @param {type:"number"}

    layout = Layout(width='550px', height='600px')
    left = Textarea(value=df.loc[index, left_column], layout=layout, description="Original:")
    right = Textarea(value=df.loc[index, right_column], layout=layout, description='Processado:')

    return Box([left, right])

In [13]:
INPUT_FOLDER = Path(userdata.get('base_set_2023'))

OUTPUT_FOLDER = Path(userdata.get('base_output'))

base_33k_filepath = INPUT_FOLDER / "acordaos_principais_33k.csv"
base_40k_filepath = INPUT_FOLDER / "acordaos_principais_40k.csv"

assert base_33k_filepath.exists()
assert base_40k_filepath.exists()

filename = base_33k_filepath
# filename = base_40k_filepath

df = pd.read_csv(filename, nrows=1_000, encoding='utf8')

df.head()

,numero_processo,id_documento,conteudo,data_hora_documento,codigos_movimentos_temas
0,10042442520168260541,54994081 - 0,== PODER JUDICIÁRIO\na TRIBUNAL DE JUSTIÇA DO ...,2018-07-17,85714
1,00272971919988260562,42752474 - 0,TRIBUNAL DE JUSTIÇA\n PODER JUDICIÁRIO\n São ...,2017-07-31,85721
2,21140481120188260000,57226790 - 0,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,2018-09-18,85471;85714
3,20924950520188260000,110484386 - 0,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,2021-11-21,85738
4,10130665020148260451,82218470 - 0,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,2020-07-18,85609;85738


In [9]:
limpo = PreProcessamentoLimpo()
normalizado = PreProcessamentoNormalizado(column_text='conteudo')

In [ ]:
# help(PreProcessamentoLimpo)
# help(PreProcessamentoNormalizado)

In [14]:
# Graças à extensão do scikit-learn, podemos utilizar o método fit_transform
limpo_33k = limpo.fit_transform(df)

limpo_33k.shape

(1000,)

In [15]:
# Ou podemos utilizar somente o método transform
normalizado_33k = normalizado.transform(limpo_33k)

normalizado_33k.shape

(1000,)

In [16]:
df['formatado_limpo'] = limpo_33k
df['formatado_normalizado'] = normalizado_33k

df.head()

,numero_processo,id_documento,conteudo,data_hora_documento,codigos_movimentos_temas,formatado_limpo,formatado_normalizado
0,10042442520168260541,54994081 - 0,== PODER JUDICIÁRIO\na TRIBUNAL DE JUSTIÇA DO ...,2018-07-17,85714,"Registro: 2018.0000521991 ACÓRDÃO Vistos, rela...",_registro_juridico_ acordao vistos relatados d...
1,00272971919988260562,42752474 - 0,TRIBUNAL DE JUSTIÇA\n PODER JUDICIÁRIO\n São ...,2017-07-31,85721,"Registro: 2017.0000544227 ACÓRDÃO Vistos, rela...",_registro_juridico_ acordao vistos relatados d...
2,21140481120188260000,57226790 - 0,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,2018-09-18,85471;85714,"Registro: 2018.0000723696 ACÓRDÃO Vistos, rela...",_registro_juridico_ acordao vistos relatados d...
3,20924950520188260000,110484386 - 0,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,2021-11-21,85738,"Registro: 2021.0000941088 ACÓRDÃO Vistos, rela...",_registro_juridico_ acordao vistos relatados d...
4,10130665020148260451,82218470 - 0,PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO EST...,2020-07-18,85609;85738,"Registro: 2020.0000546717 ACÓRDÃO Vistos, rela...",_registro_juridico_ acordao vistos relatados d...


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    PreProcessamentoLimpo(column_text='conteudo'),
    PreProcessamentoNormalizado(),
    TfidfVectorizer(max_features=3_000, ngram_range=(1, 1))
)

X = pipe.fit_transform(df)

In [18]:
pipe

Pipeline(steps=[('preprocessamentolimpo', PreProcessamentoLimpo()),
                ('preprocessamentonormalizado', PreProcessamentoNormalizado()),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_features=3000, ngram_range=(1, 2)))])

In [33]:
X

<1000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 932815 stored elements in Compressed Sparse Row format>

In [29]:
index = 500 # @param {type:"number"}

display(df, index, left_column='conteudo', right_column='formatado_limpo')

Box(children=(Textarea(value=' PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n Registro: 202…

In [30]:
index = 500 # @param {type:"number"}

display(df, index, left_column='conteudo', right_column='formatado_normalizado')

Box(children=(Textarea(value=' PODER JUDICIÁRIO\n TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n\n Registro: 202…

In [31]:
index = 500 # @param {type:"number"}

display(df, index, left_column='formatado_limpo', right_column='formatado_normalizado')

Box(children=(Textarea(value='Registro: 2022.0000113843 ACÓRDÃO Vistos, relatados e discutidos estes autos de …